## Caricamento del dataset e del tokenizer
estrazione della lunghezza dei token per ogni articolo, unione di tutte le sezioni train, validation, test.

In [1]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer

model_name = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Da un oggetto DatasetDict estraiamo un solo Dataset con tutte le sezioni che useremo
cnn_dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")
def add_token_length(batch):
    # Tokenizza il batch
    tokens = tokenizer(batch["article"], truncation=False)
    # Conta i token
    return {"article_token_len": [len(ids) for ids in tokens["input_ids"]]}
processed = cnn_dataset.map(add_token_length, batched=True, batch_size=1000)
cnn_dataset = concatenate_datasets([processed["train"], processed["validation"], processed["test"]])
print(type(cnn_dataset))

<class 'datasets.arrow_dataset.Dataset'>


## Data cleaning con pandas
eclusione della colonna id, esclusione delle righe con campi vuoti

In [2]:
import pandas as pd
# convertiamo il  Dataset in un Dataframe pandas
df_cnn_dailymail = cnn_dataset.to_pandas()
print(type(df_cnn_dailymail))

display(df_cnn_dailymail.head())
#rimuoviamo la colonna "id" che tanto non ci serve per il nostro scopo
df_cnn_dailymail = df_cnn_dailymail.drop(columns=["id"])

# rimuoviamo righe con valori vuoti
df_cnn_dailymail = df_cnn_dailymail.dropna()

display(df_cnn_dailymail.head())


<class 'pandas.core.frame.DataFrame'>


,article,highlights,id,article_token_len
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,565
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9,888
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37,919
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,531
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,1196


,article,highlights,article_token_len
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,565
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,888
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",919
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",531
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",1196


### Analisi sulle lunghezze di riassunti e articoli

In [ ]:
# nuove colonne con le lunghezze di articoli e riassunti
df_cnn_dailymail["article_len"] = df_cnn_dailymail["article"].map(lambda x: len(x.split()))
df_cnn_dailymail["highlights_len"] = df_cnn_dailymail["highlights"].map(lambda x: len(x.split()))

df_troppo_corti = df_cnn_dailymail[df_cnn_dailymail["highlights_len"]/df_cnn_dailymail["article_len"] < 1/100]
df_troppo_corti.to_html("df_troppo_corti.html", index=False)
print(df_troppo_corti.shape)




(163, 5)


,article,highlights,article_token_len,article_len,highlights_len
244504,"Downton Abbey's famously grim butler, Mr Bates .",And now for the good news . . . Following a pa...,15,8,152


2


In [12]:

display(df_cnn_dailymail.head())
count = (df_cnn_dailymail["article_token_len"] > 1024).sum()
print("Articoli oltre i 1024 token:", count)

,article,highlights,article_token_len,article_len,highlights_len
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,565,455,41
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,888,698,49
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",919,743,43
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",531,414,27
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",1196,973,44


Articoli oltre i 1024 token: 93438
